### Background

Although wildfires are a natural function of many ecosystems, climate change is leading to increase risk and severity of wildfires in numerous ways. Decreased snowpack and earlier snowmelt, drying out of wetlands, thawing of permafrost and higher air temperature all mean that larger areas are prone to drought and wildfires. Additionally, with increasing development of fire-prone areas not only are fires started more freuqently, but the toll of human lives and property damage is greater. Increasing wildfires with warming creates a positive feedback whereby increased burning, especially of peatlands, releases significant carbon dioxide to the atmosphere and drive further warming. While the main strategy to mitigate damages from wildfires is to practice fire supression, these tactics have made current ecosystems more susceptible to burning. Recently, Schoennagela et al. (2017) advocate for terminating fire supression practices and instead focus on building fire-resitant communities by managing wild and prescribed burns and consciously planning development with a full-assesment of risk.


### Sources:

Pierre-Louis, K., & Popovich, N. (2018). Climate Change Is Fueling Wildfires Nationwide. New Report Warns. The New York Times www. nytimes.com.

Schoennagel, T., Balch, J. K., Brenkert-Smith, H., Dennison, P. E., Harvey, B. J., Krawchuk, M. A., ... & Whitlock, C. (2017). Adapt to more wildfire in western North American forests as climate changes. Proceedings of the National Academy of Sciences, 114(18), 4582-4590.

Sullivan, A., Baker, E., Kurvits, T., Popescu, A., Paulson, A. K., Cardinal Christianson, A., ... & Reisen, F. (2022). Spreading like wildfire: The rising threat of extraordinary landscape fires.

In [5]:
# Import packages
import os
import warnings

import earthpy as et
import geodatasets as gds
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import pyogrio 

warnings.simplefilter('ignore')

### Watershed Boundary Dataset description

The Watershed Boundary Dataset for the US Geological Survey contains boundary data for watersheds across the United States at multiple nested scales. For this analysis we used regional hydrologic units denoted with a two-didgit code. In this analysis we evaluaate fire density within watersheds because these physiographic and hydrologic boundaries often dicate fire extent, and fires cause downstream effects. 

### Source:
US Geological Survey. (2023, October). Watershed Boundary Dataset. US Geological Survey, Department of Interior. https://www.usgs.gov/national-hydrography/watershed-boundary-dataset

U.S. Geological Survey, National Geospatial Technical Operations Center, 2023, Watershed Boundary Dataset (WBD) - USGS National Map Downloadable Data Collection: U.S. Geological Survey.

In [6]:
# Download the WBD
wbd_url = ("https://prd-tnm.s3.amazonaws.com"
           "/StagedProducts/Hydrography"
           "/WBD/National/GDB/WBD_National_GDB.zip")
wbd_dir = et.data.get_data(url=wbd_url)
wbd_dir

'/home/jovyan/earth-analytics/data/earthpy-downloads/WBD_National_GDB'

In [21]:
# Download data
wbd_path = os.path.join(wbd_dir, 'WBD_National_GDB.gdb')
wbd_hu2_gdf = gpd.read_file(wbd_path, layer='WBDHU2', from_disk=True)
# wbd_hu2_gdf

### Description:

The wildfire occurence data used in this analysis consists of a spatial databse of fire that occurred in the United States from 1992 to 2020, and were compiled from national, state and local agencies. Data fields for each fire include discovery date, location and fire size. 

### Source:

Short, Karen C. 2022. Spatial wildfire occurrence data for the United States, 1992-2020 [FPA_FOD_20221014]. 6th Edition. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2013-0009.6

In [10]:
# Download fire data
fire_url = ("https://www.fs.usda.gov/rds/archive/products"
            "/RDS-2013-0009.6/RDS-2013-0009.6_Data_Format2_GDB.zip")
fire_dir = et.data.get_data(url=fire_url)
fire_dir

Extracted output to /home/jovyan/earth-analytics/data/earthpy-downloads/RDS-2013-0009.6_Data_Format2_GDB


'/home/jovyan/earth-analytics/data/earthpy-downloads/RDS-2013-0009.6_Data_Format2_GDB'

In [22]:
# Download and read fire dataframe
fire_path = os.path.join(fire_dir, 'Data', 'FPA_FOD_20221014.gdb')
if not 'fire_gdf' in globals():
    fire_gdf = pyogrio.read_dataframe(fire_path, layer='Fires')

# fire_gdf.head()

In [23]:
# Clean up geodataframe columns
fire_clean_gdf = (
    fire_gdf
    [['FOD_ID', 'DISCOVERY_DATE', 'FIRE_SIZE', 'geometry']]
    .set_index('FOD_ID')
)
fire_clean_gdf.DISCOVERY_DATE = pd.to_datetime(fire_clean_gdf.DISCOVERY_DATE)
fire_clean_gdf = fire_clean_gdf.to_crs(wbd_hu2_gdf.crs)
# fire_clean_gdf

In [24]:
# Join geodataframes
fire_region_gdf = (
    wbd_hu2_gdf
    [['name', 'geometry']]
    .sjoin(fire_clean_gdf, how='inner', predicate='intersects')
)

# Compute maximum fire size
fire_region_gdf = (
    fire_region_gdf
    .groupby(['name', fire_region_gdf.DISCOVERY_DATE.dt.year])
    .agg(
        max_fire_size = ('FIRE_SIZE', 'max'),
        num_fires = ('index_right', 'count')
    )
)

# fire_region_gdf

In [25]:
# Compute area of regional watersheds
wbd_hu2_gdf['area_ha'] = (
    wbd_hu2_gdf.to_crs(9822).area 
    # Convert to hectares
    / 10000 
    # Convert to millions of hectares
    / 1000000
)

# Compute total fires in each watershed
fire_count_df = (
    fire_region_gdf
    .reset_index()
    [['name', 'num_fires']]
    .groupby('name')
    .sum())

# Add the area back
fire_density_gdf = (
    wbd_hu2_gdf
    .set_index('name')
    .join(fire_count_df)
)

# Compute fire density
fire_density_gdf['fire_density_per_ha'] = (
    fire_density_gdf.num_fires / fire_density_gdf.area_ha)
# fire_density_gdf.fire_density_per_ha

In [29]:
# Set ylabels and titles
labels = pd.DataFrame(dict(
    column_name = ['max_fire_size', 'num_fires'],
    ylabel = ['Fire Size (million ha)', 'Number of Fires'],
    title = ['Largest fire on record in the region', 
             'Total number of fires on record in the region']))

def fire_plot(region_name, df=fire_region_gdf, labels=labels):
    """
    Create a multi-panel plot for a region

  
    Parameters
    ----------
    region_name : str
      The name of the region to generate a plot for. Must exists 
      in the 'name' index of df.
    df : pd.DataFrame
      The dataframe with the data to plot. Columns much match
      an item in labels.column_name to be plotted
    labels : pd.DataFrame
      Plot labels. Must have a 'column_name', 'ylabel', and 'title'
      columns with str values. Each row will be a subplot.

    Returns
    -------
    plot : hv.core.layout.Layout
      A holoviews plot layout or similar. For use with hv.DynamicMap.
    """
    # Generate a subplot for each row in the labels
    subplots = []
    # Iterate through the labels row by row
    for i, labs in labels.iterrows():
        # Create subplot
        subplot = (
            df.xs(region_name, level='name')
            [[labs.column_name]]
            .hvplot(
                xlabel='Year', ylabel=labs.ylabel, title=labs.title
            ))
        # Disable x-axis scrolling
        subplot = subplot.opts()
        subplots.append(subplot)

    # Stack the subplots vertically
    plot = hv.Layout(subplots).cols(1)
    return plot

# Create a dropdown menu to switch between regions
(
    hv.DynamicMap(
        # The plotting function for the two-panel fire history
        fire_plot,
        # Define the dimension for the dropdown
        kdims=[('region', 'Region')])
    # Add the explicit indexing - region names as a bokeh dimension
    .redim.values(region=fire_region_gdf.reset_index().name)
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'628a9fd8-e394-4045-ba47-7b27bfb3bdad': {'version…

In [27]:
# simplify fire density detaframe geometry
fire_density_gdf.geometry = fire_density_gdf.geometry.simplify(tolerance=0.1)
# fire_density_gdf

In [28]:
# plot fire density data 
poly_plot = (
    gv.Polygons(
        fire_density_gdf
        .reset_index()
        [['fire_density_per_ha', 'geometry']])
    .opts(
        width=600, height=600,
        colorbar=True, color='fire_density_per_ha',
        cmap='plasma', line_color='white',
        xaxis='bare', yaxis='bare', tools=['hover'],
    )
)
gv.tile_sources.OSM * poly_plot


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (fire_density_per_ha)